# Manual Odds Entry System

This notebook allows manual entry of odds from BetFair, Pinnacle, and FanDuel, automatically fetches Polymarket prices, and saves all data with synchronized timestamps.

## Setup and Imports

In [2]:
import asyncio
import pandas as pd
import numpy as np
from datetime import datetime
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import nest_asyncio
from pathlib import Path
import os

# Allow nested asyncio in Jupyter
nest_asyncio.apply()

# Import from the installed package
from src.models.market_probability import MarketProbability
from src.services.polymarket_service import fetch_current_price
from src.calculators.american_odds import american_to_probability, probability_to_american
from src.calculators.decimal_odds import decimal_to_probability, probability_to_decimal

print("✅ All imports successful!")

ImportError: cannot import name 'american_to_probability' from 'src.calculators.american_odds' (/home/jonathanmines/Documents/code/SignalDrift/src/calculators/american_odds.py)

## Manual Odds Entry Interface

In [ ]:
class ManualOddsEntry:
    def __init__(self):
        self.data_storage = []
        self.setup_ui()
        
    def setup_ui(self):
        """Create the user interface widgets"""
        
        # Header
        self.header = widgets.HTML(
            value="<h2>📊 Manual Odds Entry System</h2>",
            layout=widgets.Layout(margin='0 0 20px 0')
        )
        
        # Market configuration
        self.market_config = widgets.VBox([
            widgets.HTML("<h3>Market Configuration</h3>"),
            widgets.HBox([
                widgets.Label("Polymarket Slug:", layout=widgets.Layout(width='150px')),
                widgets.Text(
                    placeholder="e.g., mlb-sd-mil-2025-06-08",
                    layout=widgets.Layout(width='300px')
                )
            ]),
            widgets.HBox([
                widgets.Label("Team A Name:", layout=widgets.Layout(width='150px')),
                widgets.Text(
                    placeholder="e.g., San Diego Padres",
                    layout=widgets.Layout(width='200px')
                )
            ]),
            widgets.HBox([
                widgets.Label("Team B Name:", layout=widgets.Layout(width='150px')),
                widgets.Text(
                    placeholder="e.g., Milwaukee Brewers",
                    layout=widgets.Layout(width='200px')
                )
            ])
        ])
        
        self.slug_input = self.market_config.children[1].children[1]
        self.team_a_input = self.market_config.children[2].children[1]
        self.team_b_input = self.market_config.children[3].children[1]
        
        # Odds format selector
        self.odds_format = widgets.Dropdown(
            options=['American', 'Decimal'],
            value='American',
            description='Odds Format:',
            layout=widgets.Layout(width='200px')
        )
        
        # Create odds input sections for each sportsbook
        self.betfair_section = self.create_sportsbook_section("BetFair", "#1f77b4")
        self.pinnacle_section = self.create_sportsbook_section("Pinnacle", "#ff7f0e")
        self.fanduel_section = self.create_sportsbook_section("FanDuel", "#2ca02c")
        
        # Submit button
        self.submit_button = widgets.Button(
            description="📤 Submit All Odds",
            button_style='success',
            layout=widgets.Layout(width='200px', height='50px')
        )
        self.submit_button.on_click(self.on_submit_clicked)
        
        # Status output
        self.status_output = widgets.Output()
        
        # Results display
        self.results_output = widgets.Output()
        
        # Export section
        self.export_button = widgets.Button(
            description="💾 Export to CSV",
            button_style='info',
            layout=widgets.Layout(width='150px')
        )
        self.export_button.on_click(self.export_to_csv)
        
        self.clear_button = widgets.Button(
            description="🗑️ Clear Data",
            button_style='warning',
            layout=widgets.Layout(width='150px')
        )
        self.clear_button.on_click(self.clear_data)
        
        # Layout everything
        self.main_layout = widgets.VBox([
            self.header,
            self.market_config,
            widgets.HTML("<hr>"),
            self.odds_format,
            widgets.HTML("<h3>Odds Entry</h3>"),
            widgets.HBox([self.betfair_section, self.pinnacle_section, self.fanduel_section]),
            widgets.HTML("<hr>"),
            self.submit_button,
            self.status_output,
            widgets.HTML("<hr>"),
            widgets.HBox([self.export_button, self.clear_button]),
            self.results_output
        ])
        
    def create_sportsbook_section(self, name, color):
        """Create a section for entering odds for a specific sportsbook"""
        
        style_html = f"<div style='background-color: {color}; color: white; padding: 10px; border-radius: 5px; margin-bottom: 10px;'><h4 style='margin: 0;'>{name}</h4></div>"
        
        team_a_odds = widgets.FloatText(
            description="Team A:",
            placeholder="Enter odds",
            layout=widgets.Layout(width='150px')
        )
        
        team_b_odds = widgets.FloatText(
            description="Team B:",
            placeholder="Enter odds",
            layout=widgets.Layout(width='150px')
        )
        
        section = widgets.VBox([
            widgets.HTML(style_html),
            team_a_odds,
            team_b_odds
        ], layout=widgets.Layout(margin='0 10px'))
        
        # Store references for easy access
        setattr(self, f'{name.lower()}_team_a', team_a_odds)
        setattr(self, f'{name.lower()}_team_b', team_b_odds)
        
        return section
    
    def convert_odds_to_probability(self, odds_value, odds_format):
        """Convert odds to probability based on format"""
        try:
            if odds_format == 'American':
                return american_to_probability(odds_value)
            elif odds_format == 'Decimal':
                return decimal_to_probability(odds_value)
            else:
                raise ValueError(f"Unsupported odds format: {odds_format}")
        except Exception as e:
            return None
    
    def on_submit_clicked(self, button):
        """Handle submit button click"""
        asyncio.create_task(self.submit_odds())
    
    async def submit_odds(self):
        """Submit all odds and fetch Polymarket data"""
        with self.status_output:
            clear_output(wait=True)
            print("🔄 Processing submission...")
        
        # Generate request ID (epoch time)
        request_id = int(time.time())
        submission_time = datetime.utcnow()
        
        # Validate inputs
        if not self.slug_input.value or not self.team_a_input.value or not self.team_b_input.value:
            with self.status_output:
                clear_output(wait=True)
                print("❌ Please fill in market configuration (slug and team names)")
            return
        
        entries = []
        
        # Process manual odds entries
        sportsbooks = [
            ('betfair', 'BetFair'),
            ('pinnacle', 'Pinnacle'),
            ('fanduel', 'FanDuel')
        ]
        
        for book_attr, book_name in sportsbooks:
            team_a_widget = getattr(self, f'{book_attr}_team_a')
            team_b_widget = getattr(self, f'{book_attr}_team_b')
            
            if team_a_widget.value and team_b_widget.value:
                # Convert odds to probabilities
                prob_a = self.convert_odds_to_probability(team_a_widget.value, self.odds_format.value)
                prob_b = self.convert_odds_to_probability(team_b_widget.value, self.odds_format.value)
                
                if prob_a is None or prob_b is None:
                    with self.status_output:
                        clear_output(wait=True)
                        print(f"❌ Invalid odds format for {book_name}")
                    return
                
                # Create MarketProbability instances
                entries.extend([
                    MarketProbability(
                        request_id=request_id,
                        fetched_at=submission_time,
                        probability=prob_a,
                        source=book_name.lower(),
                        team=self.team_a_input.value,
                        meta={
                            'odds_format': self.odds_format.value,
                            'original_odds': team_a_widget.value,
                            'entry_type': 'manual'
                        }
                    ),
                    MarketProbability(
                        request_id=request_id,
                        fetched_at=submission_time,
                        probability=prob_b,
                        source=book_name.lower(),
                        team=self.team_b_input.value,
                        meta={
                            'odds_format': self.odds_format.value,
                            'original_odds': team_b_widget.value,
                            'entry_type': 'manual'
                        }
                    )
                ])
        
        # Fetch Polymarket data
        with self.status_output:
            clear_output(wait=True)
            print("🔄 Fetching Polymarket data...")
        
        try:
            polymarket_data = fetch_current_price(self.slug_input.value)
            
            if polymarket_data:
                # Extract team prices from polymarket response
                team_a_name = self.team_a_input.value
                team_b_name = self.team_b_input.value
                
                # Find matching team prices
                team_a_price = None
                team_b_price = None
                
                for team_name, price in polymarket_data.items():
                    if team_name not in ['fetched_at', 'updated_at']:
                        if team_name.lower() in team_a_name.lower() or team_a_name.lower() in team_name.lower():
                            team_a_price = price
                        elif team_name.lower() in team_b_name.lower() or team_b_name.lower() in team_name.lower():
                            team_b_price = price
                
                # If we couldn't match by name, use the first two prices
                if team_a_price is None or team_b_price is None:
                    prices = [v for k, v in polymarket_data.items() if k not in ['fetched_at', 'updated_at']]
                    if len(prices) >= 2:
                        team_a_price, team_b_price = prices[0], prices[1]
                
                if team_a_price is not None and team_b_price is not None:
                    entries.extend([
                        MarketProbability(
                            request_id=request_id,
                            fetched_at=polymarket_data['fetched_at'],
                            probability=team_a_price,
                            source='polymarket',
                            team=team_a_name,
                            updated_at=polymarket_data.get('updated_at'),
                            meta={
                                'slug': self.slug_input.value,
                                'entry_type': 'api'
                            }
                        ),
                        MarketProbability(
                            request_id=request_id,
                            fetched_at=polymarket_data['fetched_at'],
                            probability=team_b_price,
                            source='polymarket',
                            team=team_b_name,
                            updated_at=polymarket_data.get('updated_at'),
                            meta={
                                'slug': self.slug_input.value,
                                'entry_type': 'api'
                            }
                        )
                    ])
                
        except Exception as e:
            with self.status_output:
                clear_output(wait=True)
                print(f"⚠️ Failed to fetch Polymarket data: {e}")
        
        # Store all entries
        self.data_storage.extend(entries)
        
        # Clear form
        self.clear_form()
        
        # Update display
        await self.update_results_display()
        
        with self.status_output:
            clear_output(wait=True)
            print(f"✅ Successfully submitted {len(entries)} entries with request ID: {request_id}")
    
    def clear_form(self):
        """Clear the odds entry form"""
        for book in ['betfair', 'pinnacle', 'fanduel']:
            getattr(self, f'{book}_team_a').value = 0.0
            getattr(self, f'{book}_team_b').value = 0.0
    
    async def update_results_display(self):
        """Update the results display with current data"""
        with self.results_output:
            clear_output(wait=True)
            
            if not self.data_storage:
                print("No data entries yet.")
                return
            
            # Convert to DataFrame for display
            df_data = []
            for entry in self.data_storage:
                df_data.append({
                    'Request ID': entry.request_id,
                    'Timestamp': entry.fetched_at.strftime('%Y-%m-%d %H:%M:%S'),
                    'Source': entry.source,
                    'Team': entry.team,
                    'Probability': f"{entry.probability:.4f}",
                    'Entry Type': entry.meta.get('entry_type', 'unknown')
                })
            
            df = pd.DataFrame(df_data)
            
            # Group by request ID for better display
            print(f"📊 Current Data ({len(self.data_storage)} entries):")
            print("\n")
            
            for request_id in df['Request ID'].unique():
                group = df[df['Request ID'] == request_id]
                print(f"🔗 Request ID: {request_id} ({len(group)} entries)")
                
                # Calculate spread for this group
                if len(group) >= 2:
                    probs = [float(p) for p in group['Probability']]
                    spread = max(probs) - min(probs)
                    print(f"   📈 Spread: {spread:.4f}")
                
                print(group.to_string(index=False))
                print("\n")
    
    def export_to_csv(self, button):
        """Export data to CSV file"""
        if not self.data_storage:
            with self.status_output:
                clear_output(wait=True)
                print("❌ No data to export")
            return
        
        # Create data directory if it doesn't exist
        data_dir = Path('../data')
        data_dir.mkdir(exist_ok=True)
        
        # Create filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = data_dir / f'manual_odds_entries_{timestamp}.csv'
        
        # Convert to DataFrame
        df_data = []
        for entry in self.data_storage:
            row = {
                'request_id': entry.request_id,
                'fetched_at': entry.fetched_at.isoformat(),
                'updated_at': entry.updated_at.isoformat() if entry.updated_at else None,
                'probability': entry.probability,
                'source': entry.source,
                'team': entry.team,
                **{f'meta_{k}': v for k, v in entry.meta.items()}
            }
            df_data.append(row)
        
        df = pd.DataFrame(df_data)
        df.to_csv(filename, index=False)
        
        with self.status_output:
            clear_output(wait=True)
            print(f"✅ Exported {len(df)} entries to {filename}")
    
    def clear_data(self, button):
        """Clear all stored data"""
        self.data_storage.clear()
        asyncio.create_task(self.update_results_display())
        
        with self.status_output:
            clear_output(wait=True)
            print("🗑️ All data cleared")
    
    def display(self):
        """Display the interface"""
        display(self.main_layout)

# Create the interface
odds_entry = ManualOddsEntry()
print("✅ Manual Odds Entry interface created!")

## Launch the Interface

In [ ]:
# Display the odds entry interface
odds_entry.display()

## Data Analysis Helper Functions

In [ ]:
def analyze_spreads():
    """Analyze spreads across all submissions"""
    if not odds_entry.data_storage:
        print("No data to analyze")
        return
    
    # Group by request_id
    request_groups = {}
    for entry in odds_entry.data_storage:
        if entry.request_id not in request_groups:
            request_groups[entry.request_id] = []
        request_groups[entry.request_id].append(entry)
    
    print("📈 Spread Analysis:")
    print("=" * 50)
    
    spreads = []
    for request_id, entries in request_groups.items():
        if len(entries) >= 2:
            probs = [e.probability for e in entries]
            spread = max(probs) - min(probs)
            spreads.append(spread)
            
            timestamp = entries[0].fetched_at.strftime('%H:%M:%S')
            sources = list(set([e.source for e in entries]))
            
            print(f"Request {request_id} ({timestamp})")
            print(f"  Sources: {', '.join(sources)}")
            print(f"  Spread: {spread:.4f}")
            print(f"  Min prob: {min(probs):.4f}, Max prob: {max(probs):.4f}")
            print()
    
    if spreads:
        print(f"Average spread: {np.mean(spreads):.4f}")
        print(f"Max spread: {max(spreads):.4f}")
        print(f"Min spread: {min(spreads):.4f}")

def get_latest_submission():
    """Get the most recent submission data"""
    if not odds_entry.data_storage:
        print("No data available")
        return None
    
    latest_request_id = max([e.request_id for e in odds_entry.data_storage])
    latest_entries = [e for e in odds_entry.data_storage if e.request_id == latest_request_id]
    
    print(f"📊 Latest Submission (Request ID: {latest_request_id}):")
    print("=" * 50)
    
    for entry in latest_entries:
        print(f"{entry.source:12} | {entry.team:20} | {entry.probability:.4f}")
    
    return latest_entries

def export_summary_stats():
    """Export summary statistics"""
    if not odds_entry.data_storage:
        print("No data to analyze")
        return
    
    # Create summary DataFrame
    summary_data = []
    
    # Group by request_id
    request_groups = {}
    for entry in odds_entry.data_storage:
        if entry.request_id not in request_groups:
            request_groups[entry.request_id] = []
        request_groups[entry.request_id].append(entry)
    
    for request_id, entries in request_groups.items():
        if len(entries) >= 2:
            probs = [e.probability for e in entries]
            sources = [e.source for e in entries]
            timestamp = entries[0].fetched_at
            
            summary_data.append({
                'request_id': request_id,
                'timestamp': timestamp,
                'num_sources': len(set(sources)),
                'min_probability': min(probs),
                'max_probability': max(probs),
                'spread': max(probs) - min(probs),
                'avg_probability': np.mean(probs),
                'sources': ', '.join(set(sources))
            })
    
    df_summary = pd.DataFrame(summary_data)
    
    # Create data directory if it doesn't exist
    data_dir = Path('../data')
    data_dir.mkdir(exist_ok=True)
    
    # Export
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = data_dir / f'odds_summary_{timestamp}.csv'
    df_summary.to_csv(filename, index=False)
    
    print(f"✅ Summary exported to {filename}")
    print(f"📊 Summary Statistics:")
    print(df_summary.describe())
    
    return df_summary

print("✅ Analysis functions ready!")

## Quick Analysis Tools

In [1]:
# Quick analysis buttons
analyze_button = widgets.Button(description="📈 Analyze Spreads", button_style='info')
latest_button = widgets.Button(description="📊 Latest Submission", button_style='info')
summary_button = widgets.Button(description="📋 Export Summary", button_style='info')

analyze_button.on_click(lambda b: analyze_spreads())
latest_button.on_click(lambda b: get_latest_submission())
summary_button.on_click(lambda b: export_summary_stats())

analysis_tools = widgets.HBox([analyze_button, latest_button, summary_button])
display(analysis_tools)

print("✅ Analysis tools ready!")

NameError: name 'widgets' is not defined

## Usage Instructions

### How to Use This Interface:

1. **Market Configuration**:
   - Enter the Polymarket slug (e.g., `mlb-sd-mil-2025-06-08`)
   - Enter Team A and Team B names

2. **Select Odds Format**:
   - Choose American (e.g., -110, +150) or Decimal (e.g., 1.91, 2.50)

3. **Enter Odds**:
   - Fill in odds for any combination of BetFair, Pinnacle, and FanDuel
   - You don't need to fill all sources - submit what you have

4. **Submit**:
   - Click "📤 Submit All Odds"
   - The system will automatically fetch current Polymarket prices
   - All entries will be grouped with the same request ID (timestamp)

5. **Export Data**:
   - Use "💾 Export to CSV" to save all data
   - Use analysis tools for quick insights

### Data Structure:
Each submission creates MarketProbability instances for:
- Each team × each manual source entered (up to 6 entries)
- Each team × Polymarket (2 additional entries)
- All grouped by request_id for easy analysis

### Features:
- ✅ Real-time odds conversion (American ↔ Decimal ↔ Probability)
- ✅ Automatic Polymarket price fetching
- ✅ Synchronized timestamps with request grouping
- ✅ Spread analysis and arbitrage detection
- ✅ CSV export with comprehensive metadata
- ✅ Live data preview and analysis tools